In [5]:
# import library
import multiprocessing.dummy
import sys
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from btrack.utils import segmentation_to_objects
from scipy.ndimage.filters import gaussian_laplace, minimum_filter
from skimage import data, img_as_float
from skimage.exposure import equalize_adapthist
from skimage.transform import pyramid_expand, pyramid_gaussian, pyramid_reduce
from skimage.segmentation import watershed,expand_labels
from skimage.morphology import remove_small_objects
from skimage.io import imsave
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'last_expr'
import time
from tqdm import tqdm, trange
sys.path.append("..")
import imgio
import ipywidgets as ipyw
# %matplotlib inline
# %matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import seg
import tensorflow as tf
from cflow import gradient_flow_3d
from cflow.blob import find_spheres, sphere_log,peakfilter
from imgio import napari_base
from ipywidgets import interactive
from seg import segment, track
from strutensor.hessian import StructureEvi, enhance_ridges
from utils import npixel
from utils.measure import label_statics
from utils import file_base
import os
from imgio import plot
from skimage.filters.thresholding import (threshold_isodata, threshold_li,
                                          threshold_mean, threshold_minimum,
                                          threshold_multiotsu,
                                          threshold_niblack, threshold_otsu,
                                          threshold_triangle, threshold_yen,threshold_local)
from cflow.localthreadhold import local_threshold
methods = [threshold_isodata, threshold_li,
            threshold_mean,
            threshold_minimum,
            threshold_multiotsu,
            threshold_niblack, threshold_otsu,
            threshold_triangle, threshold_yen,local_threshold]
from skimage.filters import threshold_yen,threshold_otsu,threshold_yen


In [6]:
#basic fun
#   classify function #
#-----------------------#
# svm function
from utils import measure
def predict_cls(clff,xs):
    clf,pca,scaler,_=clff
    xs=np.array([x.ravel() for x  in xs])
    xs=scaler.transform(xs)
    xs=pca.transform(xs)
    y_pred = clf.predict(xs)
    return y_pred
def filter_p(points,lables):
    pps=[]
    pps2=[]
    for p,l in zip(points,lables):
        if l>0:
            pps.append(p)
        else:
            pps2.append(p)
    return pps,pps2


def filter_p3d(points,image,clfs):
    #boxsize=clfs[-1]
    imagemax=np.max(image,axis=0)
    dpsimg=segment.cropfromseed(imagemax,points[...,1:],boxsize=clfs[-1])
    y_pred=predict_cls(clfs,dpsimg)
    # print(y_pred)
    points=filter_p(points,y_pred)
    return points

    
def reshape_data(XX):
    X=[]
    for x in XX:
        # x=filters.sobel(x)
        x=x.reshape(16,16,1)
        x=x.astype(np.float32)
        X.append(x)
    # print("img shape :",X[0].shape)
    return np.array(X)
def model_predict(points,image,modle,boxsize):
    if image.ndim==3:
        image=np.max(image,axis=0)
        #modle.summary()
        dpsimg=segment.cropfromseed(image,points[...,1:],boxsize=boxsize)
    elif image.ndim==2:
        dpsimg=segment.cropfromseed(image,points,boxsize=boxsize)
    
    # for im in dpsimg:
    #     plt.figure()
    #     plt.imshow(im)
    dpsimg=reshape_data(dpsimg)
    y_p=modle.predict(dpsimg)
    y_pred=y_p.argmax(axis=-1)
    points,points2=filter_p(points,y_pred)
    return points,points2
# lmodel=load_default_model()

def show_img_info(images):
    shape=images.shape
    dtype=images.dtype
    ndim=images.ndim
    print("image shape :",shape)
    print("image dtype :",dtype)
    print("image ndim :",ndim)


def remove_small_lable(mask,thsize):
    labs=measure.unique_labs(mask)
    newmask=np.zeros_like(mask)
    for lab in labs:
        mm=mask==lab
        mask2=remove_small_objects(mm,thsize)
        if not mask.any():
            mask2=mm
        newmask[mask2]=lab
    return newmask   
def remove_small_lables(masks,thsize):
    newmasks=[]
    masks=np.array(masks)
    print("shape ",masks.shape)
    n_frames=imgs.shape[0]
    for i in tqdm(range(n_frames), desc='Processing'):
        mask=masks[i]
        labs=measure.unique_labs(mask)
        newmask=np.zeros_like(mask)
        for lab in labs:
            mm=mask==lab
            mask2=remove_small_objects(mm,thsize)
            if not mask2.any():
                mask2=mm
            newmask[mask2]=lab
        newmasks.append(newmask)
    return np.array(newmasks)
def th_img(imgs,ls,binary_func):
    global_th=binary_func(imgs[0])
    lss=np.zeros_like(imgs,dtype=np.uint16)

    for i in range(len(imgs)):
        img=imgs[i]
        adth=img>global_th
        # denmask,_=predict.predict_single_img(unetmodel,img)
        l=ls*adth
        l[l<2]=0
        l=remove_small_lable(l,6)
        lss[i]=l
        # lss[i][denmask==1]=0
    lss[lss<2]=0
       
    return lss


def predict_single_img(model,img):
    # print("img shape : ",img.shape)
    cm=2**model.pool_num
    ndim=img.ndim
    shape=img.shape
    padd=[(0,(cm-s%cm)%8) for s in img.shape]
    if ndim==3:
        padd[0]=(0,0)
    img=np.pad(img,padd)
    img=np.expand_dims(img,axis=-1).astype(np.float32)
    if ndim==2:
        img=np.expand_dims(img,axis=0).astype(np.float32)
    imgns=[]
    spineprs=[]
    denprs=[]
    # print(img.shape)
    for im in img:
        im=np.expand_dims(im,axis=0).astype(np.float32)
        ppr=model.predict(im)[0]
        # pr = ppr.argmax(axis=-1)
        imgn = ppr.argmax(axis=-1)
        spineprs.append(ppr[...,2])
        denprs.append(ppr[...,1])
        imgns.append(imgn)
    imgns=np.array(imgns)
    imgns=np.squeeze(imgns)
    spineprs=np.array(spineprs)
    spineprs=np.squeeze(spineprs)
    denprs=np.array(denprs)
    denprs=np.squeeze(denprs)
    # print(shape,imgns.shape)
    
    obj=[slice(0,s) for s in shape ]
    return imgns[obj],spineprs[obj],denprs[obj]
def modify_mask(mask,projmask,sizeth=20):
    denmask=mask==1
    spinemask=mask==2
    denmask=remove_small_objects(denmask,min_size=sizeth)
    projmask=remove_small_objects(projmask==1,min_size=sizeth)
    # print(mask.shape[0])
    projmask=np.tile(projmask,(mask.shape[0],1,1))
    spinemask[projmask>0]=0
    mask=spinemask*2+denmask
    return mask
    
def label_instance(img,corner,spinemask):
    labels,num=segment.ndilable(corner,1)
    ls = segment.foreach_grow(img, num_iter=4, 
                                init_level_set=labels,
                                searchbox=[7,7,7],
                                sizeth=maxspinesize,adth=spinemask,
                                method="geo",smoothing=0)
    spine_label=watershed(-img,ls,mask=spinemask)
    return spine_label
def predict_time_img(model,imgs):
    # print("img shape : ",img.shape)
    masks=[]
    spine_prs=[]
    den_prs=[]
    n_frames=imgs.shape[0]
    for i in tqdm(range(n_frames), desc='Processing'):
        img=imgs[i]
        mask,spineprs,denprs=predict_single_img(model,img)
        projmask,_,_=predict_single_img(model,np.max(img,axis=0))
        mask=modify_mask(mask,projmask,20)
        masks.append(mask)
        spine_prs.append(spineprs)
        den_prs.append(denprs)
    masks=np.array(masks)
    masks=np.squeeze(masks)    
    spine_prs=np.array(spine_prs)
    spine_prs=np.squeeze(spine_prs) 
    den_prs=np.array(den_prs)
    den_prs=np.squeeze(den_prs) 
    return masks,spine_prs,den_prs

def spine_proj(masks):
    projmask=np.max(masks==2,axis=0)
    return projmask

def adth_func_time(imgs,binary_func):
    th=binary_func(imgs[0])
    mean1 = np.mean(imgs[0][imgs[0]<th])
    adths=[img>th for img in imgs]
    return np.array(adths),th,mean1
def adth_func(img,binary_func):
    th=binary_func(img)
    return img>th

def remove_den(imgs,masks):
    mean1 = np.mean(imgs[0]*(masks[0]==0))
    # stdv1 = np.std(imgs[0]*(masks[0]==0))
    imgns=imgs.copy()
    imgns[masks==1]=mean1
    return imgns
def keep_spine(imgs,masks):
    mean1 = np.mean(imgs[0]*(masks[0]==0))
    # stdv1 = np.std(imgs[0]*(masks[0]==0))
    imgns=imgs.copy()
    imgns[masks!=2]=mean1
    return imgns
def keep_spine_by_prob(imgs,prob,th=0.5,cval=140):
    # mean1 = np.mean(imgs[0]*(prob>th))
    # stdv1 = np.std(imgs[0]*(masks[0]==0))
    imgns=imgs.copy()
    imgns[prob<th]=cval
    return imgns
def keep_spine_proj(imgs,masks):
    spines=keep_spine(imgs,masks)
    return np.max(spines,axis=0)



In [7]:
# super params
#-----------------------#
#   paras  #
#-----------------------#

steps=2 # caculate stick gradient step lenth 1-4, larger more error , smaller than spine diameter
sigma=(2,2) # z,y,x
lambda1=1
lambda2=1.5 # lambda2 need large than lambda1
sigma_hessian = 4
sigma_supp = 100
countth=10*20 # fiber largest pixels, include z stack, that to say lenth*stackn
smallobj=200
spine_radius=35 # for iteration to growth area
maxspinesize=800
spotrange=[3,11]
binary_func=threshold_li
use_sobel=False
search_box=[7,25,25]
keep_time=5
peak_use_gaussian=True

untmodel=None

In [29]:
# load data
#-----------------------#
#   data loader  #
#-----------------------#
filename=r"D:\spine\Train\4D\deconvolve_4D\decon_20211111-24D.tif"
#-----------------------#
#   Output file path  #
#-----------------------#
ddir,shotname,suffix=file_base.split_filename(filename)
savedir=r"D:\spine\Train\4D\deconvolve_4D\statistic"
dendir=r"D:\spine\Train\4D\deconvolve_4D\remove_den"
roidir=r"D:\spine\Train\4D\deconvolve_4D\roi_proj"
predictdir=r"D:\spine\Train\4D\deconvolve_4D\predict"
areadir=os.path.join(savedir,"area")
meandir=os.path.join(savedir,"mean")
file_base.create_dir(areadir)
file_base.create_dir(meandir)
file_base.create_dir(dendir)
file_base.create_dir(roidir)
file_base.create_dir(predictdir)

areafile=os.path.join(areadir,shotname+"_area.csv")
proj_areafile=os.path.join(areadir,shotname+"_proj_area.csv")
meanfile=os.path.join(meandir,shotname+"_mean_inten.csv")
proj_meanfile=os.path.join(meandir,shotname+"_proj_meaninten.csv")
sumfile=os.path.join(meandir,shotname+"_sum_inten.csv")
proj_sumfile=os.path.join(meandir,shotname+"_proj_suminten.csv")

removedenfile=os.path.join(dendir,shotname+"_remove_den.tif")

proj_roifile=os.path.join(roidir,shotname+"_roi_proj.tif")
each_roifile=os.path.join(roidir,shotname+"_roi.tif")

maskfile=os.path.join(predictdir,shotname+"_mask.tif")
spineprfile=os.path.join(predictdir,shotname+"_spine_pr.tif")
denprfile=os.path.join(predictdir,shotname+"_den_pr.tif")
#-----------------------#
#   model loader  #
#-----------------------#
import pickle
from net.networks import unet
from net.trainers import predict
#---  spot filter model---#
# fclf=r"D:\spine\segment\some_analysis_for_spine_img\clf.pickle"
# with open(fclf, 'rb') as f:
#     clff = pickle.load(f)
#---  dendrite filter model---#
unetmodel=unet.UNet2D()
# D:\spine\segment\some_analysis_for_spine_img\models\model\unet_model\M2d_den\modelep200-loss0.010 copy.h5
# D:\spine\segment\some_analysis_for_spine_img\models\model\unet_model\M2d_den\modelep082-loss0.012 copy.h5
modelpath=r"D:\spine\segment\some_analysis_for_spine_img\models\model\unet_model\M2d_den\modelep200-loss0.010 copy.h5"
unetmodel.build(input_shape =(4,512,512,1))
unetmodel.load_weights(modelpath)
peakmodel=tf.keras.models.load_model(
    r"D:\spine\segment\some_analysis_for_spine_img\models\model\cnn_model", custom_objects=None, compile=True, options=None
)

imgs = imgio.ski_imread(filename)
show_img_info(imgs)

image shape : (60, 11, 512, 512)
image dtype : float32
image ndim : 4


# PART ONE
track by max projection ,maybe stable


1. Let's see one img

In [30]:
#--- BINARY---#
img=imgs[0]
adth=adth_func(img,binary_func)

#--- unet model predict spine,dendrite and background---#
mask,spineprs,_=predict_single_img(unetmodel,img)
projmask,s_,d_=predict_single_img(unetmodel,np.max(img,axis=0))
mask=modify_mask(mask,projmask,20)
spinemask=mask==2
corner=peakfilter(spineprs,5,0,use_gaussian=True)*spinemask
spine_label=label_instance(img,corner,spinemask)
spine_label=spine_label*adth

In [31]:
# view single frame segment
viewer=plot.imshow3d(img,[s_,d_],label_lay=[mask,spine_label],label_lay_name=["lls","spine"])

Exception in callback BaseAsyncIOLoop._handle_events(15020, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(15020, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData

2. run all img

In [9]:
# precit all frame
all_mask,all_spine_pr,all_den_pr=predict_time_img(unetmodel,imgs)

Processing:   0%|          | 0/50 [00:00<?, ?it/s]C:\Users\ZLY\AppData\Local\Temp\ipykernel_22424\2216229698.py:148: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return imgns[obj],spineprs[obj],denprs[obj]
Processing: 100%|██████████| 50/50 [01:10<00:00,  1.41s/it]


In [11]:
# view multi frames segment
keep_spine_imgs=keep_spine_by_prob(imgs,all_spine_pr,th=0.5)#keep_spine(imgs,all_mask)
adths,th,mean1=adth_func_time(imgs,binary_func)

"""illustration
imgs
"spine prob" : predict spine probability
"adth img spine" : img of spine area , masked by threshold mask and spine mask
"all segmnet" : predict classification result
"""
viewer=plot.imshow3d(imgs,
                              [all_spine_pr,keep_spine_imgs*adths],
                              name_Is=["spine prob","adth img spine"],
                              label_lay=[all_mask],
                              label_lay_name=["all segmnet"])


Exception in callback BaseAsyncIOLoop._handle_events(7800, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(7800, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData\R

3. edit proj mask

In [12]:
#-----------------------#
#   GET projection mask label  #
#-----------------------#

from skimage import morphology
structure=np.ones((3,)*3)
spine_mask_proj=morphology.binary_erosion(np.max(all_mask==2,axis=0),structure)
# ---------You can use img projection peak , just pr projection seem better by little test
# ----use img projection
# img_spine_proj=keep_spine_proj(imgs*adths,all_mask)
# img_corner=peakfilter(img_spine_proj,3,0,use_gaussian=True)*spine_mask_proj
# spine_label_img=label_instance(img_spine_proj,img_corner,spine_mask_proj)

# ----use prob projection
# pr_adths=adth_func_time(all_spine_pr,binary_func)
# pr_adths_proj=np.max(pr_adths,axis=0)
pr_spine_proj=keep_spine_proj(all_spine_pr*adths,all_mask)
# find proj peak
pr_corner=peakfilter(pr_spine_proj,3,0,use_gaussian=True)*spine_mask_proj
spine_label_pr=label_instance(pr_spine_proj,pr_corner,np.max(all_mask==2,axis=0))

# viewer=plot.imshow3d(imgs,
#                               [img_spine_proj,pr_spine_proj,img_corner,pr_corner],
#                               name_Is=["spine proj","spine prob proj","peak of img","peak of prob"],
#                               label_lay=[spine_label_img,spine_label_pr],
#                               lablel_lay_name=["img_label","pr label"])


In [13]:
# see and edit result
"""illustration
imgs
"pr_spine_proj" : predict spine probability' projection mask by threshold mask
"pr_corner"     : peak of pr_spine_proj
"spine_label_pr" : spine instance segment projection
"all segmnet" : predict classification result
You need edit lable ( remove/merge some error segment,and so on)
"""
viewer=plot.imshow3d(imgs,
                              [keep_spine_by_prob(imgs,all_spine_pr,th=0.4),all_spine_pr],
                              name_Is=["spine imgs","spine prob proj"],
                              label_lay=[np.array([spine_label_pr for im in imgs])],
                              label_lay_name=["pr label"])

edit_pr_spine_proj=viewer.layers["pr label"].data[0]

Exception in callback BaseAsyncIOLoop._handle_events(9456, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(9456, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData\R

In [4]:
# resee
spine_label_pr=edit_pr_spine_proj
# viewer=plot.imshow3d(keep_spine_by_prob(imgs,all_spine_pr,th=0.4),
#                               [imgs,all_spine_pr],
#                               name_Is=["img","spine prob proj"],
#                               label_lay=[np.array([spine_label_pr for im in imgs])],
#                               label_lay_name=["pr label"])


NameError: name 'edit_pr_spine_proj' is not defined

In [3]:
edit_pr_spine_proj.shape

NameError: name 'edit_pr_spine_proj' is not defined

In [20]:
edit_pr_spine_proj_t=edit_pr_spine_proj.copy()
# edit_pr_spine_proj_t=imgio.ski_imread(proj_roifile)
edit_pr_spine_proj_t,_=segment.resortseg(edit_pr_spine_proj_t,2)
adths,th,mean1=adth_func_time(imgs,binary_func)
proj_spine_masks=np.array([edit_pr_spine_proj_t for im in imgs])
spine_maskes=(all_spine_pr>0.4)*adths*proj_spine_masks
# nspine_maskes=remove_small_lables(spine_maskes,6)
# nspine_maskes=remove_small_lables(spine_maskes,6)
spine_imgs=keep_spine_by_prob(imgs,all_spine_pr,th=0.4,cval=mean1)

In [21]:
viewer=plot.imshow3d(imgs,
                              [all_spine_pr,spine_imgs],
                              name_Is=["spine prob","spine img"],
                              label_lay=[spine_maskes,edit_pr_spine_proj_t],
                              label_lay_name=["spine area label","spine area proj"])

Exception in callback BaseAsyncIOLoop._handle_events(11048, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(11048, 1)>
Traceback (most recent call last):
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "c:\Users\ZLY\AppData\Local\Programs\Python\Python38\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\ZLY\AppData

4. save mask

In [22]:
#save roi mask
import tifffile

#edit_pr_spine_proj_t : spine max mask (same size for each frame)
imsave(proj_roifile,edit_pr_spine_proj_t.astype(np.uint16))
# imsave(maskfile,all_mask)
# imsave(spineprfile,all_spine_pr)

# all_mask : den spine back mask
tifffile.imwrite(
    maskfile,
    all_mask.astype(np.uint16),
    imagej=True,
    photometric='minisblack',
    metadata={'axes': 'TZYX'},
)

# all_spine_pr : spine probability
tifffile.imwrite(
    spineprfile,
    all_spine_pr,
    imagej=True,
    photometric='minisblack',
    metadata={'axes': 'TZYX'},
)
tifffile.imwrite(
    denprfile,
    all_den_pr,
    imagej=True,
    photometric='minisblack',
    metadata={'axes': 'TZYX'},
)

# nspine_maskes : spine's area mask
tifffile.imwrite(
    each_roifile,
    spine_maskes.astype(np.uint16),
    imagej=True,
    photometric='minisblack',
    metadata={'axes': 'TZYX'},
)

# removedenfile : remove den area imgs
tifffile.imwrite(
    removedenfile,
    spine_imgs,
    imagej=True,
    photometric='minisblack',
    metadata={'axes': 'TZYX'},
)

D:\spine\Train\4D\deconvolve_4D\roi_proj\decon_20211112-14D_roi_proj.tif is a low contrast image


# Part Two 

1. load data

In [23]:
same_spine_maskes=imgio.ski_imread(proj_roifile)
all_mask=imgio.ski_imread(maskfile)
all_spine_pr=imgio.ski_imread(spineprfile)
all_den_pr=imgio.ski_imread(denprfile)
spine_maskes=imgio.ski_imread(each_roifile)
spine_imgs=imgio.ski_imread(removedenfile)
# spine_imgs=imgs.copy()
# spine_imgs[all_den_pr>0.4]=mean1

In [50]:
viewer=plot.imshow3d(imgs,
                              [all_spine_pr,spine_imgs],
                              name_Is=["spine prob","spine img"],
                              label_lay=[spine_maskes,edit_pr_spine_proj_t],
                              label_lay_name=["spine area label","spine area proj"])

2. caculate statistic

In [24]:
import pandas as pd
same_spine_maskes_t=np.array([same_spine_maskes])
same_spine_maskes_ts=np.array([same_spine_maskes for im in imgs])

arr=measure.label_series_statics(same_spine_maskes_t,same_spine_maskes_t,measure="area")
arr_volume_same=arr#*0.16*0.16*0.3

arr=measure.label_series_statics(imgs,spine_maskes,measure="area")
df_volume=arr*0.16*0.16*0.3
df_volume=pd.DataFrame(df_volume)

arr=measure.label_series_statics(spine_imgs,same_spine_maskes_ts,measure="intensity_mean")
backvs=[np.mean(im[ms==0]) for im,ms in zip(imgs,all_mask)]
arr[:,0]=backvs
arr=np.row_stack((arr_volume_same,arr))
df_mean=pd.DataFrame(arr)

# proj information statics
spine_maskes_proj=np.max(spine_maskes,axis=1)
spine_imgs_proj=np.max(spine_imgs,axis=1)
imgs_proj=np.max(imgs,axis=1)
all_mask_proj=np.max(all_mask,axis=1)
same_spine_maskes_t_proj=np.max(same_spine_maskes_t,axis=1)
same_spine_maskes_ts_proj=np.max(same_spine_maskes_ts,axis=1)
arr=measure.label_series_statics(imgs_proj,same_spine_maskes_t_proj,measure="area")
arr_area_same=arr#*0.16*0.16


arr=measure.label_series_statics(spine_maskes_proj,spine_maskes_proj,measure="area")
df_proj_area=arr*0.16*0.16
df_proj_area=pd.DataFrame(df_proj_area)


arr=measure.label_series_statics(spine_imgs_proj,same_spine_maskes_ts_proj,measure="intensity_mean")
backvs=[np.mean(im[ms==0]) for im,ms in zip(imgs_proj,all_mask_proj)]
arr[:,0]=backvs
arr=np.row_stack((arr_area_same,arr))
df_proj_mean=pd.DataFrame(arr)




In [25]:
# save csv result
df_volume.to_csv(areafile)
df_mean.to_csv(meanfile)
df_proj_area.to_csv(proj_areafile)
df_proj_mean.to_csv(proj_meanfile)


In [127]:
del df_volume
del df_mean
del df_proj_area
del df_proj_mean
del arr
del same_spine_maskes_ts_proj

# Part Three   

compare before deconvolve

In [ ]:

def nms(image,hitpeak,returnmask=True):
    # if clabs is None:
    #     clabs=list(np.unique(hitpeak))
    # if 0 in clabs:
    #     clabs.remove(0)
    indexs=np.argwhere(hitpeak>0)
    indexs=list(indexs)
    values=image[tuple(zip(*indexs))]
    vlabs=hitpeak[tuple(zip(*indexs))]
    zipped_pairs = zip(values,vlabs,indexs)
    #sorted_pairs = sorted(zipped_pairs,lambda x :x[2])
    zipped_pairs = list(zip(vlabs,values,indexs))
    zipped_pairs.sort(key=lambda x:(x[0],-x[1]))
    nmspoints=[]
    viewedlabs=[]
    for lab,value,ind in zipped_pairs:
        if lab in viewedlabs: continue
        else:
            viewedlabs.append(lab)
            nmspoints.append(ind)
    if returnmask:
        mask=np.zeros_like(image)
        mask[tuple(zip(*nmspoints))]=1
        return mask
    else:
        return nmspoints

def keep_lable_seg(segment,labs):
    ls=np.zeros_like(segment)
    for lab in labs:
        ls[segment==lab]=lab
    return ls


def track_by_seg(image,segment,
                 radius,num_iter,searchbox,
                 keep_time=3,resvers=None,labcounts={},
                sizeth=np.inf,method="geo",adth=None,
                smoothing=0,lambda1=1, lambda2=3):
    if segment is None:
        return
    segment=segment.copy()
    if resvers is not None:
        segment[segment==0]=resvers[segment==0]
        
    adth=local_threshold(image)
    peakmask=peakfilter(image,radius)*adth
    hitpeak=segment*peakmask
    
    prelab=set(np.unique(segment))
    curlab=set(np.unique(hitpeak))
    
    #intersec
    interseclab=prelab&curlab
    
    #nmsmask=nms(image,hitpeak,returnmask=True)*hitpeak
    #print(np.unique(nmsmask))
    intersecls=seg.segment.foreach_grow_area(image,num_iter,hitpeak,
                     searchbox=searchbox,sizeth=sizeth,
                     adth=adth,method=method,smoothing=smoothing,
                     lambda1=lambda1,lambda2=lambda2)
        
    #diff # which not find
    diflab=prelab-curlab
    nofinders=keep_lable_seg(segment,diflab)
    for lab in diflab:
        if lab not in labcounts:
            labcounts[lab]=1
        elif labcounts[lab]>keep_time:
            nofinders[nofinders==lab]=0
        else:
            labcounts[lab]+=1
    
    return intersecls,peakmask,nofinders